# Introduction / Business Problem

Chicago has been declining in population for the past 3 years. It's rare for a metropolitan area to have a net decrease in population year-over-year. Here is more information about the decline: https://www.chicagotribune.com/news/local/breaking/ct-met-chicago-cook-county-population-20180320-story.html  
  
The types of venues that are present in a given zip code, among other types of information available from foursquare, may serve as good predictors of whether that zip code is likely to experience population decline. My goal for this project is to make a classifier for a given zip code to predict if it will experience decline or not. This kind of analysis could be helpful to demographers, sociologists, city planners, politicians, economists, neighborhood associations, and probably a few more professions.

# Data

On the data, I plan to get the venues from the explore API from foursquare for each zip code in Chicago. I will use this data for features. Then I plan to use the total population for each of the zip codes from the US Census Bureau American Fact Finder tool to determine if there was decline between the years 2016 and 2017.  
  
Here is a list of all chicago zip codes: http://www.city-data.com/zipmaps/Chicago-Illinois.html  
Here is the link to the census bureau data that I will import as a two CSV files, one for 2016 and one for 2017: https://factfinder.census.gov/bkmk/table/1.0/en/ACS/17_5YR/B01003/8600000US60007|8600000US60018|8600000US60068|8600000US60106|8600000US60131|8600000US60176|8600000US60601|8600000US60602|8600000US60603|8600000US60604|8600000US60605|8600000US60606|8600000US60607|8600000US60608|8600000US60609|8600000US60610|8600000US60611|8600000US60612|8600000US60613|8600000US60614|8600000US60615|8600000US60616|8600000US60617|8600000US60618|8600000US60619|8600000US60620|8600000US60621|8600000US60622|8600000US60623|8600000US60624|8600000US60625|8600000US60626|8600000US60628|8600000US60629|8600000US60630|8600000US60631|8600000US60632|8600000US60633|8600000US60634|8600000US60636|8600000US60637|8600000US60638|8600000US60639|8600000US60640|8600000US60641|8600000US60642|8600000US60643|8600000US60644|8600000US60645|8600000US60646|8600000US60647|8600000US60649|8600000US60651|8600000US60652|8600000US60653|8600000US60654|8600000US60655|8600000US60656|8600000US60657|8600000US60659|8600000US60660|8600000US60661|8600000US60706|8600000US60707|8600000US60714|8600000US60804|8600000US60827

Use autogenerated code to import the csv files from IBM Cloud Object Storage. df_data_1 is for 2016. df_data_2 is for 2017.

In [6]:
# The code was removed by Watson Studio for sharing.

,GEO.id,GEO.id2,GEO.display-label,Estimate; Total,Margin of Error; Total
0,8600000US60007,60007,ZCTA5 60007,33733,161
1,8600000US60018,60018,ZCTA5 60018,30519,816
2,8600000US60068,60068,ZCTA5 60068,37567,323
3,8600000US60106,60106,ZCTA5 60106,20215,328
4,8600000US60131,60131,ZCTA5 60131,18072,168


In [7]:
body = client_0830bd8539294ecb8e07bb128b7fed36.get_object(Bucket='capstonesegmentingandsplitting-donotdelete-pr-uzmj8rdmck50ke',Key='ACS_17_5YR_B01003.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_2 = pd.read_csv(body)
df_data_2.head()

,GEO.id,GEO.id2,GEO.display-label,Estimate; Total,Margin of Error; Total
0,8600000US60007,60007,ZCTA5 60007,33691,191
1,8600000US60018,60018,ZCTA5 60018,30955,996
2,8600000US60068,60068,ZCTA5 60068,37810,312
3,8600000US60106,60106,ZCTA5 60106,20300,342
4,8600000US60131,60131,ZCTA5 60131,18070,129


Clean up the data into one dataframe.

In [8]:
population_16 = df_data_1[['GEO.id2','Estimate; Total']]
population_16.columns = ['Zip Code', 'Population 2016']
population_16.head()

,Zip Code,Population 2016
0,60007,33733
1,60018,30519
2,60068,37567
3,60106,20215
4,60131,18072


In [9]:
population_17 = df_data_2[['GEO.id2','Estimate; Total']]
population_17.columns = ['Zip Code','Population 2017']
population_17.head()

,Zip Code,Population 2017
0,60007,33691
1,60018,30955
2,60068,37810
3,60106,20300
4,60131,18070


In [10]:
population = population_16
population['Population 2017'] = population_17['Population 2017']
population.head()

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,Zip Code,Population 2016,Population 2017
0,60007,33733,33691
1,60018,30519,30955
2,60068,37567,37810
3,60106,20215,20300
4,60131,18072,18070


In [11]:
population['Decline'] = population['Population 2016'] - population['Population 2017'] > 0
population.head()

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,Zip Code,Population 2016,Population 2017,Decline
0,60007,33733,33691,True
1,60018,30519,30955,False
2,60068,37567,37810,False
3,60106,20215,20300,False
4,60131,18072,18070,True


Here is an example of the explore API from foursquare:

In [12]:
import requests

CLIENT_ID = 'GC0PBW0SRCSK1CRI3F4DJVH0MMFHZX3R1MXJCWDHJLD4TACG' # your Foursquare ID
CLIENT_SECRET = 'HGJ5IRVIOMNM0OCAURIDLFJJBKWJW4LCKCHNRRJ21Q3KWSWL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
radius = 2000
LIMIT = 100
test_lat= 40.1038
test_long= -83.0200
name = 'Worthington'

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            test_lat, 
            test_long, 
            radius, 
            LIMIT)

# make the GET request
results = requests.get(url).json()["response"]['groups'][0]['items']
venues_list=[]
# return only relevant information for each nearby venue
venues_list.append([(
            name, 
            test_lat, 
            test_long, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
nearby_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Worthington,40.1038,-83.02,"Pies & Pints - Worthington, OH",40.109009,-83.018817,Pizza Place
1,Worthington,40.1038,-83.02,J Liu Restaurant & Bar,40.104396,-83.016468,Restaurant
2,Worthington,40.1038,-83.02,Amano's,40.109415,-83.019863,Pizza Place
3,Worthington,40.1038,-83.02,Worthington Winter Farmers Market,40.108385,-83.019309,Market
4,Worthington,40.1038,-83.02,Blaze Pizza,40.107374,-83.015947,Pizza Place


# Methodology

The first task is to get the latitudes and longitudes for each of the zip codes. There was an approach to do this in the week 3 assignment with geocoder but it kept hanging for me. I searched for the coordinates of each one and saved the results in a csv file and imported into this notebook from IBM Cloud Storage. You are able to search individually for the lat-long coordinates from the google front-end ironically.

In [13]:
body = client_0830bd8539294ecb8e07bb128b7fed36.get_object(Bucket='capstonesegmentingandsplitting-donotdelete-pr-uzmj8rdmck50ke',Key='ZipCode Coords.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_3 = pd.read_csv(body)
df_data_3.head()

,Zip Code,Latitude,Longitude
0,60007,42.0114,-88.0021
1,60018,42.0145,-87.8992
2,60068,42.0085,-87.8452
3,60106,41.9509,-87.9433
4,60131,41.9349,-87.8795


Use this function that was used in the lab exercise to get nearby venues for all of the zip codes.

In [14]:
def getNearbyVenues(zipcodes, latitudes, longitudes):
    radius=3000
    LIMIT=1000
    venues_list=[]
    for zipcode, lat, lng in zip(zipcodes, latitudes, longitudes):
        print(zipcode)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            500)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            zipcode, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Zip Code', 
                  'Zip Code Latitude', 
                  'Zip Code Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
chicago_venues = getNearbyVenues(zipcodes = df_data_3['Zip Code'], latitudes = df_data_3['Latitude'],longitudes = df_data_3['Longitude'])

60007
60018
60068
60106
60131
60176
60601
60602
60603
60604
60605
60606
60607
60608
60609
60610
60611
60612
60613
60614
60615
60616
60617
60618
60619
60620
60621
60622
60623
60624
60625
60626
60628
60629
60630
60631
60632
60633
60634
60636
60637
60638
60639
60640
60641
60642
60643
60644
60645
60646
60647
60649
60651
60652
60653
60654
60655
60656
60657
60659
60660
60661
60706
60707
60714
60804
60827


In [16]:
print(chicago_venues.shape)

(6452, 7)


Counts per zip code:

In [17]:
chicago_venues.groupby('Zip Code').count()

,Zip Code Latitude,Zip Code Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Zip Code,,,,,,
60007,72,72,72,72,72,72
60018,100,100,100,100,100,100
60068,100,100,100,100,100,100
60106,79,79,79,79,79,79
60131,100,100,100,100,100,100
60176,100,100,100,100,100,100
60601,100,100,100,100,100,100
60602,100,100,100,100,100,100
60603,100,100,100,100,100,100


Now make a one hot encoding for the Venue Categories and get the mean of all the categories for each zip code. These will be used as features.

In [21]:
chicago_onehot = pd.get_dummies(chicago_venues[['Venue Category']], prefix = "", prefix_sep = "")
chicago_onehot['Zip Code'] = chicago_venues['Zip Code'] 
chicago_onehot.head()

,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,...,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit,Zip Code
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,60007
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,60007
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,60007
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60007
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,60007


In [22]:
chicago_grouped = chicago_onehot.groupby('Zip Code').mean().reset_index()
chicago_grouped.head()

,Zip Code,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Waterfront,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,60007,0.00,0.0,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.027778,...,0.0,0.0,0.0,0.0,0.0,0.013889,0.0,0.00,0.0,0.0
1,60018,0.02,0.0,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.020000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.0
2,60068,0.01,0.0,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.030000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.01,0.0,0.0
3,60106,0.00,0.0,0.0,0.0,0.012658,0.00,0.012658,0.025316,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.0
4,60131,0.00,0.0,0.0,0.0,0.000000,0.01,0.000000,0.000000,0.040000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.0


Get the features X from the chicago_grouped dataframe:

In [23]:
X = chicago_grouped.drop(['Zip Code'],axis=1)
X.head()

,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,...,Waterfront,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,0.00,0.0,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.027778,0.0,...,0.0,0.0,0.0,0.0,0.0,0.013889,0.0,0.00,0.0,0.0
1,0.02,0.0,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.020000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.0
2,0.01,0.0,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.030000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.01,0.0,0.0
3,0.00,0.0,0.0,0.0,0.012658,0.00,0.012658,0.025316,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.0
4,0.00,0.0,0.0,0.0,0.000000,0.01,0.000000,0.000000,0.040000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.0


Now normalize the features:

In [27]:
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)
X

array([[-0.40866778, -0.21650635, -0.20359464, ..., -0.70467267,
        -0.12309149, -0.12309149],
       [ 2.69517271, -0.21650635, -0.20359464, ..., -0.70467267,
        -0.12309149, -0.12309149],
       [ 1.14325247, -0.21650635, -0.20359464, ...,  0.683947  ,
        -0.12309149, -0.12309149],
       ..., 
       [-0.40866778, -0.21650635, -0.20359464, ..., -0.70467267,
        -0.12309149, -0.12309149],
       [ 1.14325247, -0.21650635, -0.20359464, ..., -0.70467267,
        -0.12309149, -0.12309149],
       [-0.40866778, -0.21650635, -0.20359464, ..., -0.70467267,
        -0.12309149, -0.12309149]])

Recall that the labels are the 'Decline' column from the population dataframe. Set this series to y:

In [32]:
y=population['Decline']
y.head()

0     True
1    False
2    False
3    False
4     True
Name: Decline, dtype: bool

Split the data into training and testing sets:

In [80]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.4, random_state=2)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

Train set: (40, 330) (40,)
Test set: (27, 330) (27,)


Use LogisticRegression from scikitlearn to make one classifier:

In [81]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(C=0.001, solver='liblinear').fit(X_train,y_train)

Use SVM from scikitlearn to make another classifier:

In [95]:
from sklearn import svm
svm_classifier = svm.SVC(kernel = 'linear')
svm_classifier.fit(X_train, y_train)
yhat_SVM = svm_classifier.predict(X_test)

# Results

The next task is to evaluate the performance of the models:

In [97]:
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss

yhat_lr = LR.predict(X_test)
yhat_prob = LR.predict_proba(X_test)
print('For LinearRegression:')
print('jaccard score: ',jaccard_similarity_score(y_test, yhat_lr))
print('f1 score: ', f1_score(y_test, yhat_lr, average='weighted'))
from sklearn.metrics import log_loss
print('log loss: ',log_loss(y_test, yhat_prob))
print('')
print('For SVM:')
print('jaccard score: ',jaccard_similarity_score(y_test, yhat_SVM))
print('f1 score: ',f1_score(y_test, yhat_SVM, average='weighted'))

For LinearRegression:
jaccard score:  0.592592592593
f1 score:  0.55828460039
log loss:  0.68407870582

For SVM:
jaccard score:  0.592592592593
f1 score:  0.589220945083


# Discussion

As can be seen in the evaluation metrics in the results section, the models aren't that great at prediction but they're not completely off either. The Jaccard scores are higher than 0.5 which means there are more similar entries between the test values (y_test) and the predictions (yhat_lr or yhat_SVM) than dissimilar. Also, the f1 score is closer to a perfect model (value of 1) than the worst model (value of 0). The log loss is high for the logistic regression model, as a perfect model has a value of 0.  
  
The foursquare explore API is the source of venues which are turned into the features used here. Two parameters in the API that could be changed to see improvements are limit and radius. Limit appears to cut off at 100. This can be observed in the cell marked as "counts by zip code" above. If more venues were returned, then there would be more accurate venue averages for features. As for radius, I'm not sure how much area each of these zip codes occupy. Therefore, the value used here perhaps had the results for each zip code overlapping or not covering enough of the actual zip code areas.

# Conclusion

This analysis may not have made perfect models for predicting, but both the LinearRegression and SVM models performed better than random guessing. There are other features that could be used. A lot of demographic tables available from the Census Bureau like income brackets and age are good examples. The label could be predicting party preference in elections. These kinds of regression models could be used to predict food deserts or those areas most likely to be affected by the opioid epidemic. There are a lot of neat applications for data science in helping to work on and fix social concerns.